# Statistics

This notebook shows show the do some basic statistics on Neuropixels `ChannelMap` and its blueprint.

## Number of selected electrodes

For a channelmap

In [5]:
from neurocarto.probe_npx import ChannelMap

chmap = ChannelMap(24)
len(chmap) # empty map

0

In [10]:
chmap.add_electrode((0,0,0))
len(chmap)

1

For a blueprint

In [11]:
from neurocarto.probe_npx import NpxProbeDesp

D = NpxProbeDesp()

# all electrodes, chmap here used as a probe type reference, we do not use its content inside.
blueprint = D.all_electrodes(chmap)
len([e for e in blueprint if e.state == D.STATE_USED])

0

In [13]:
# Get selected electrodes
# This function will read the content of chmap
len(D.all_channels(chmap))

1

## Number of electrode set with full-density category

In [16]:
blueprint = D.load_blueprint('Fig3_example.blueprint.npy', chmap)

len([e for e in blueprint if e.category == D.CATE_FULL])

66

## Area Efficiency

we define the area efficiency $Aeff$ of a given blueprint $Q$ and its outcomes channel map $S$ as

$Aeff(S|Q) = \cfrac{selected(S|Q)}{request(Q)}$, $0$ if $request(Q)=0$ 

Where $selected$ and $request$ are defined below:

In [18]:
def selected(S, Q):
    """How many electrodes are selected in density-category."""
    # we do not use S here, because Q contains the selecting information.
    v1 = len([e for e in Q if e.state == D.STATE_USED and e.category == D.CATE_SET])
    v2 = len([e for e in Q if e.state == D.STATE_USED and e.category == D.CATE_FULL])
    v3 = len([e for e in Q if e.state == D.STATE_USED and e.category == D.CATE_HALF])
    v4 = len([e for e in Q if e.state == D.STATE_USED and e.category == D.CATE_QUARTER])
    return v1 + v2 + v3 + v4

def request(Q):
    """How many electrodes are requested in density-category."""
    v1 = len([e for e in Q if e.category == D.CATE_SET])
    v2 = len([e for e in Q if e.category == D.CATE_FULL])
    v3 = len([e for e in Q if e.category == D.CATE_HALF])
    v4 = len([e for e in Q if e.category == D.CATE_QUARTER])
    return v1 + v2 + v3/2 + v4/4

When $Aeff < 1$ , it means area are not well used on average. When $Aeff > 1$ , it means area are over-used on average.

## Channel Efficiency

We define the channel efficiency $Ceff$ of a given blueprint $Q$ and its outcomes channel map $S$ as

$Ceff(S|Q) = min \{ Aeff(S|Q), \cfrac{1}{Aeff(S|Q)} \}$, $0$ if $Aeff(S|Q) = 0$

We define the channel efficiency $Ceff$ of a blueprint $Q$ as

$Ceff(Q) = max \{ Ceff(S|Q) | S \in \mathcal{S} \}$

where $\mathcal{S}$ is all possible outcomes from the given blueprint $Q$.